In [208]:
import pandas as pd

In [209]:
# Функция сохранения заголовка формата VEP
def make_header(file_name):
    header_list = []
    with open(file_name, "r") as file:
        while True:
            line = file.readline()
            if not line:
                break
            if line.split()[0] == "#Uploaded_variation": # сохраним заголовок
                header_list.append(line.split())
                return header_list[0]

In [210]:
# Чтение VEP файла
vep_file_path = 'DS44625.vep.vcf'
header_list = make_header(vep_file_path)
vep_df = pd.read_table(vep_file_path, header=None, comment='#')
vep_df.columns = header_list

# Вывод первых нескольких строк данных
vep_df.head()

,#Uploaded_variation,Location,Allele,Gene,Feature,Feature_type,Consequence,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,Extra
0,CM000663.2_46248360_A/C,CM000663.2:46248360,C,ENSG00000085999,ENST00000371975,Transcript,5_prime_UTR_variant,598,-,-,-,-,-,IMPACT=MODIFIER;STRAND=1;VARIANT_CLASS=SNV;SYM...
1,CM000663.2_46248360_A/C,CM000663.2:46248360,C,ENSG00000085999,ENST00000442598,Transcript,5_prime_UTR_variant,69,-,-,-,-,-,IMPACT=MODIFIER;STRAND=1;VARIANT_CLASS=SNV;SYM...
2,CM000663.2_46248360_A/C,CM000663.2:46248360,C,ENSG00000085999,ENST00000463715,Transcript,intron_variant,-,-,-,-,-,-,IMPACT=MODIFIER;STRAND=1;FLAGS=cds_end_NF;VARI...
3,CM000663.2_46248360_A/C,CM000663.2:46248360,C,ENSG00000085999,ENST00000469835,Transcript,intron_variant,-,-,-,-,-,-,IMPACT=MODIFIER;STRAND=1;FLAGS=cds_end_NF;VARI...
4,CM000663.2_46248360_A/C,CM000663.2:46248360,C,ENSG00000085999,ENST00000487700,Transcript,upstream_gene_variant,-,-,-,-,-,-,IMPACT=MODIFIER;DISTANCE=152;STRAND=1;VARIANT_...


In [241]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#vep_df['Consequence']
vep_df['Extra'][0]


'IMPACT=MODIFIER;STRAND=1;VARIANT_CLASS=SNV;SYMBOL=RAD54L;SYMBOL_SOURCE=HGNC;HGNC_ID=HGNC:9826;BIOTYPE=protein_coding;CANONICAL=YES;MANE_SELECT=NM_003579.4;TSL=1;APPRIS=P1;CCDS=CCDS532.1;ENSP=ENSP00000361043;SWISSPROT=Q92698.176;UNIPARC=UPI0000378007;GENE_PHENO=1;EXON=1/18;HGVSc=ENST00000371975.9:c.-46A>C'

In [212]:
vep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11592 entries, 0 to 11591
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   #Uploaded_variation  11592 non-null  object
 1   Location             11592 non-null  object
 2   Allele               11592 non-null  object
 3   Gene                 11592 non-null  object
 4   Feature              11592 non-null  object
 5   Feature_type         11592 non-null  object
 6   Consequence          11592 non-null  object
 7   cDNA_position        11592 non-null  object
 8   CDS_position         11592 non-null  object
 9   Protein_position     11592 non-null  object
 10  Amino_acids          11592 non-null  object
 11  Codons               11592 non-null  object
 12  Existing_variation   11592 non-null  object
 13  Extra                11592 non-null  object
dtypes: object(14)
memory usage: 1.2+ MB


In [213]:
# Словарь для перевода названий хромосом
chromosome_dict_hg38 = {
    "CM000663.2": "chr1",
    "CM000664.2": "chr2",
    "CM000665.2": "chr3",
    "CM000666.2": "chr4",
    "CM000667.2": "chr5",
    "CM000668.2": "chr6",
    "CM000669.2": "chr7",
    "CM000670.2": "chr8",
    "CM000671.2": "chr9",
    "CM000672.2": "chr10",
    "CM000673.2": "chr11",
    "CM000674.2": "chr12",
    "CM000675.2": "chr13",
    "CM000676.2": "chr14",
    "CM000677.2": "chr15",
    "CM000678.2": "chr16",
    "CM000679.2": "chr17",
    "CM000680.2": "chr18",
    "CM000681.2": "chr19",
    "CM000682.2": "chr20",
    "CM000683.2": "chr21",
    "CM000684.2": "chr22",
    "CM000685.2": "chrX",
    "CM000686.2": "chrY"
}

def translate_chromosome(cm_id):
    return chromosome_dict_hg38.get(cm_id, "Unknown chromosome")

In [214]:
# Функция для извлечения гена и генетического варианта из строки 'Extra'
def extract_gene_variant(extra_info):
    extra_dict = dict(item.split('=') for item in extra_info.split(';') if '=' in item)
    gene = extra_dict.get('SYMBOL', '')
    genetic_variant = extra_dict.get('HGVSc', '')
    if genetic_variant:
        genetic_variant = genetic_variant.split(':', 1)[-1]
    return pd.Series([gene, genetic_variant], index=['ГЕН', 'ГЕНЕТИЧЕСКИЙ ВАРИАНТ'])


In [215]:
# Итератор, который обрабатывает DataFrame построчно
def gene_variant_iterator(df):
    for index, row in df.iterrows():
        gene, genetic_variant = extract_gene_variant(row['Extra'])
        location = row['Location'].split(':', 1)[0]
        location = translate_chromosome(location)
        feature = row['Feature']
        yield {'ГЕН': gene, 'ПОЗИЦИЯ': location, 'ГЕНЕТИЧЕСКИЙ ВАРИАНТ': genetic_variant, 'ТРАНСКРИПТ': feature, }

In [216]:
# Сбор всех строк в список
rows = list(gene_variant_iterator(vep_df))

# Создание DataFrame из списка строк
result_df = pd.DataFrame(rows)

result_df.head()

,ГЕН,ПОЗИЦИЯ,ГЕНЕТИЧЕСКИЙ ВАРИАНТ,ТРАНСКРИПТ
0,RAD54L,chr1,c.-46A>C,ENST00000371975
1,RAD54L,chr1,c.-46A>C,ENST00000442598
2,RAD54L,chr1,c.-569-152A>C,ENST00000463715
3,RAD54L,chr1,c.-7-39A>C,ENST00000469835
4,RAD54L,chr1,,ENST00000487700


In [217]:
## Критерии ACMG
acmg_criteria = {
    'population': ['PM2', 'BA1', 'BS1', 'BS2'],
    'odds_ratio': ['PS4'],
    'phenotype': ['PP4'],
    'segregation': ['PP1', 'BS4'],
    'de_novo': ['PS2', 'PM6'],
    'cis_trans': ['PM3', 'BP2'],
    'functional': ['PS3', 'BS3'],
    'truncation': ['PVS1', 'PM4', 'BP3'],
    'amino_acid_substitution': ['PS1', 'PM5', 'PM1', 'PP2', 'BP1'],
    'in_silico_predictions': ['PP3', 'BP4', 'BP7'],
    'literature': ['BP5', 'PP5', 'BP6']
}

Добавим пока что случайные комбинации классификаций по разным признакам

In [222]:
import random
import numpy as np
# Функция для создания случайной комбинации из списка
def generate_random_combination(criteria_list):
    if random.random() < 0.7:
        return np.nan
    return random.choice(criteria_list)

# Создание нового столбца с случайными комбинациями
result_df['random_ACMG_combination'] = result_df.apply(lambda row: {
    category: generate_random_combination(acmg_criteria[category]) for category in acmg_criteria
}, axis=1)

# Вывод первых нескольких строк датафрейма
result_df.head()
#result_df.iloc[0]['random_ACMG_combination']

,ГЕН,ПОЗИЦИЯ,ГЕНЕТИЧЕСКИЙ ВАРИАНТ,ТРАНСКРИПТ,random_ACMG_combination
0,RAD54L,chr1,c.-46A>C,ENST00000371975,"{'population': 'BA1', 'odds_ratio': 'PS4', 'ph..."
1,RAD54L,chr1,c.-46A>C,ENST00000442598,"{'population': 'BA1', 'odds_ratio': nan, 'phen..."
2,RAD54L,chr1,c.-569-152A>C,ENST00000463715,"{'population': nan, 'odds_ratio': nan, 'phenot..."
3,RAD54L,chr1,c.-7-39A>C,ENST00000469835,"{'population': nan, 'odds_ratio': nan, 'phenot..."
4,RAD54L,chr1,,ENST00000487700,"{'population': nan, 'odds_ratio': 'PS4', 'phen..."


Дальше идет GPT code 
Надо всё сделать вручную

In [198]:
def has_criterion(combination, criterion):
    return criterion in combination.values()

def count_strong_criteria(combination):
    return sum(criterion.startswith('PS') for criterion in combination.values())

def count_moderate_criteria(combination):
    return sum(criterion.startswith('PM') for criterion in combination.values())

def count_supporting_criteria(combination):
    return sum(criterion.startswith('PP') for criterion in combination.values())

def count_very_strong_criteria(combination):
    return 'PVS1' in combination.values()

def count_likely_pathogenic_criteria(combination):
    return 'PM1' in combination.values()

def count_benign_criteria(combination):
    return any(criterion.startswith('BS') for criterion in combination.values())

def count_likely_benign_criteria(combination):
    return any(criterion.startswith('BP') for criterion in combination.values())


In [239]:
import numpy as np

def classify_acmg(combination):
    # Проверка на патогенность
    if isinstance(combination, dict):
        if ('PVS1' in combination.values() and
            (sum(criterion.startswith('PS') for criterion in combination.values() if isinstance(criterion, str)) >= 1 or
             sum(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)) >= 2 or
             (sum(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)) >= 1 and
              sum(criterion.startswith('PP') for criterion in combination.values() if isinstance(criterion, str)) >= 1) or
             sum(criterion.startswith('PP') for criterion in combination.values() if isinstance(criterion, str)) >= 2)):
            return 'Pathogenic'

        if (sum(criterion.startswith('PS') for criterion in combination.values() if isinstance(criterion, str)) >= 2 or
            ('PS' in combination.values() and
             (sum(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)) >= 3 or
              (sum(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)) >= 2 and
               sum(criterion.startswith('PP') for criterion in combination.values() if isinstance(criterion, str)) >= 2) or
              (sum(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)) >= 1 and
               sum(criterion.startswith('PP') for criterion in combination.values() if isinstance(criterion, str)) >= 4)))):
            return 'Likely pathogenic'

        # Проверка на вероятность патогенности
        if (('PVS1' in combination.values() and 'PM1' in combination.values()) or
            ('PS' in combination.values() and
             (sum(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)) in [1, 2] and
              sum(criterion.startswith('PP') for criterion in combination.values() if isinstance(criterion, str)) >= 2) or
             sum(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)) >= 3 or
             (sum(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)) >= 2 and
              sum(criterion.startswith('PP') for criterion in combination.values() if isinstance(criterion, str)) >= 2) or
             (sum(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)) >= 1 and
              sum(criterion.startswith('PP') for criterion in combination.values() if isinstance(criterion, str)) >= 4))):
            return 'Likely pathogenic'

        # Проверка на неопределенное значение
        if all(criterion is np.nan for criterion in combination.values()) or \
           (any(criterion.startswith('BS') for criterion in combination.values() if isinstance(criterion, str)) and
            any(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)) or
            (any(criterion.startswith('PP') for criterion in combination.values() if isinstance(criterion, str)) and
             any(criterion.startswith('PM') for criterion in combination.values() if isinstance(criterion, str)))):
            return 'Uncertain significance'

        # Проверка на вероятность доброкачественности
        if (('PS' in combination.values() and 'BP' in combination.values()) or
            (sum(criterion.startswith('BP') for criterion in combination.values() if isinstance(criterion, str)) >= 2)):
            return 'Likely benign'

        # Проверка на доброкачественное значение
        if ('BA1' in combination.values() or
            (sum(criterion.startswith('BS') for criterion in combination.values() if isinstance(criterion, str)) >= 2)):
            return 'Benign'

    return np.nan  # Если не подходит ни одно из условий или тип не является dict

# Применение функции к датафрейму
result_df['ACMG_classification'] = result_df['random_ACMG_combination'].apply(classify_acmg)


In [242]:
result_df.head()

,ГЕН,ПОЗИЦИЯ,ГЕНЕТИЧЕСКИЙ ВАРИАНТ,ТРАНСКРИПТ,random_ACMG_combination,ACMG_classification
0,RAD54L,chr1,c.-46A>C,ENST00000371975,"{'population': 'BA1', 'odds_ratio': 'PS4', 'ph...",Pathogenic
1,RAD54L,chr1,c.-46A>C,ENST00000442598,"{'population': 'BA1', 'odds_ratio': nan, 'phen...",Uncertain significance
2,RAD54L,chr1,c.-569-152A>C,ENST00000463715,"{'population': nan, 'odds_ratio': nan, 'phenot...",NaN
3,RAD54L,chr1,c.-7-39A>C,ENST00000469835,"{'population': nan, 'odds_ratio': nan, 'phenot...",Uncertain significance
4,RAD54L,chr1,,ENST00000487700,"{'population': nan, 'odds_ratio': 'PS4', 'phen...",Likely pathogenic
